<a href="https://colab.research.google.com/github/22f3002718/llama2_7b/blob/main/llama2_7b_breakpoints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install llama-stack

In [ ]:
from huggingface_hub import notebook_login
notebook_login()  # This will prompt you to paste your Hugging Face token


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

prompt = "What is the Capital of India"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate output
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        top_k=50
    )

# Decode and print the result
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"\nGenerated Text:\n{generated_text}")

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]


Generated Text:
What is the Capital of India?
Delhi is the capital of India and it is also the largest city of the country. It is located in the northern part of India and is known for its rich culture and history.
The city has been the seat of power for many dynasties and empires over the centuries. Delhi is home to some of the most famous landmarks in the country, such as the Red Fort, Jama Masjid, Qutub Minar, and Humayun’s Tomb.
Delhi is a bustling city with a population of over 16 million people. It is a hub of business, education, culture, and entertainment. The city is also home to some of the most prestigious


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

model.eval()

lengths = [512, 1024, 2048, 3072, 4096, 5000]  # Test beyond expected limits

for length in lengths:
    text = "This is a test sentence. " * (length // 5)  # approximate token count
    inputs = tokenizer(text, return_tensors="pt", truncation=False).to("cuda")

    try:
        with torch.no_grad():
            outputs = model(**inputs, use_cache=False)  # disable KV cache to get actual limit
        print(f"Length {length}: SUCCESS ✅")
    except Exception as e:
        print(f"Length {length}: FAILURE ❌ - {str(e)}")
        break  # stop at first failure


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Length 512: SUCCESS ✅
Length 1024: SUCCESS ✅
Length 2048: SUCCESS ✅
Length 3072: SUCCESS ✅
Length 4096: SUCCESS ✅
Length 5000: SUCCESS ✅


In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

model.eval()

texts = [
    "How was your day.",  # simple, positive
    "Delhi is the capital of India.",  # factual
    "asdkfj asldkfjaslkdf jalsdkfj qwerty zxcvbn...",  # gibberish
]

# Loop through each input prompt
for text in texts:
    print("="*60)
    print(f"Prompt: {text}")

    # Tokenize and send input to GPU
    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    # Generate continuation from the model
    with torch.no_grad():
        logits = model(**inputs).logits[:, -1, :]  # logits for last token (optional)
        output_ids = model.generate(**inputs, max_new_tokens=50, do_sample=True)

    # Calculate confidence from softmax
    probs = F.softmax(logits, dim=-1)
    confidence = probs.max().item()
    status = "SUCCESS ✅" if confidence > 0.6 else "FAILURE ❌"

    # Decode the generated output
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Print results
    print(f"{status} - Confidence: {confidence:.2f}")
    print("📝 Generated Output:")
    print(generated_text)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Prompt: How was your day.
FAILURE ❌ - Confidence: 0.15
📝 Generated Output:
How was your day. How did you get to work. What is the first thing you do when you get to work. How was your day. How was your day. How was your day. How was your day. How was your day. How was your day.
Prompt: Delhi is the capital of India.
FAILURE ❌ - Confidence: 0.27
📝 Generated Output:
Delhi is the capital of India. Delhi is a city of many faces, and the capital of India is no exception. The city is the seat of power, and as such, it is home to some of the most important people in the country. It is also home to some
Prompt: asdkfj asldkfjaslkdf jalsdkfj qwerty zxcvbn...
SUCCESS ✅ - Confidence: 0.66
📝 Generated Output:
asdkfj asldkfjaslkdf jalsdkfj qwerty zxcvbn...

import pytest

from pytest_mock import MockerFixture

from . import mocks


@pytest.fixture
def mock_config():
    with MockerFixture(mocks)


In [ ]:
import time
for length in [512, 1024, 2048, 3072, 4096]:
    text = "Test sentence. " * (length // 2)
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    start = time.time()
    with torch.no_grad():
        _ = model(**inputs)
    end = time.time()
    print(f"{length} tokens → ⏱️ {end - start:.2f}s")


512 tokens → ⏱️ 0.85s


OutOfMemoryError: CUDA out of memory. Tried to allocate 34.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 6.12 MiB is free. Process 8870 has 14.73 GiB memory in use. Of the allocated memory 14.13 GiB is allocated by PyTorch, and 484.96 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)